In [1]:
import pandas as pd
import numpy as np
import math

ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
m = len(movies)
tt = movies.genres

genres_list = []
item_prof = []
for i in range(0, m):
    aa = tt.iloc[i].split('|')
    genres_list += aa
    item_prof.append(aa)

In [4]:
genres_list[0:9]

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Adventure',
 'Children',
 'Fantasy',
 'Comedy']

In [5]:
item_prof[0:4]

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 ['Adventure', 'Children', 'Fantasy'],
 ['Comedy', 'Romance'],
 ['Comedy', 'Drama', 'Romance']]

In [6]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_arr = te.fit_transform(item_prof)
te_arr = te_arr.astype('int')
df = pd.DataFrame(te_arr, columns=te.columns_)
df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
movieID = movies.movieId[0:m]
df.index = movieID
df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
ratings_mean_cnt = pd.DataFrame(ratings.groupby('movieId')['rating'].sum())
ratings_mean_cnt

,rating
movieId,
1,843.0
2,377.5
3,169.5
4,16.5
5,150.5
...,...
193581,4.0
193583,3.5
193585,3.5


In [9]:
ratings_mean_cnt.sum()

rating    353083.0
dtype: float64

In [10]:
843.0/353083.0

0.0023875406065995814

In [11]:
user_movie = [["Drama", "Thriller"], ["Adventure", "Comedy", "Crime", "Romance"], ["Comedy"], ["Comedy"]]
user_rating = [6, 10, 8, 8]

te = TransactionEncoder()
te_arr = te.fit_transform(user_movie)
te_arr = te_arr.astype('int')
user_df = pd.DataFrame(te_arr, columns=te.columns_)
user_df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
0,0,0,0,1,0,1
1,1,1,1,0,1,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0


In [12]:
for i in range(4):
    user_df.iloc[i] *= user_rating[i]

user_df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
0,0,0,0,6,0,6
1,10,10,10,0,10,0
2,0,8,0,0,0,0
3,0,8,0,0,0,0


In [13]:
features = (user_df.sum()/user_df.sum().sum())
features

Adventure    0.147059
Comedy       0.382353
Crime        0.147059
Drama        0.088235
Romance      0.147059
Thriller     0.088235
dtype: float64

In [14]:
_df = df[user_df.columns]
_df

,Adventure,Comedy,Crime,Drama,Romance,Thriller
movieId,,,,,,
1,1,1,0,0,0,0
2,1,0,0,0,0,0
3,0,1,0,0,1,0
4,0,1,0,1,1,0
5,0,1,0,0,0,0
...,...,...,...,...,...,...
193581,0,1,0,0,0,0
193583,0,1,0,0,0,0
193585,0,0,0,1,0,0


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

lst = cosine_similarity([features], _df.iloc[0:m])[0]
lst

array([0.78633365, 0.30890103, 0.78633365, ..., 0.18534062, 0.        ,
       0.80314268])

In [16]:
arr = []
for i in range(len(lst)):
    arr.append((lst[i], i))

arr = sorted(arr)[::-1]
arr[:5]

[(0.8649228885013017, 7033),
 (0.8649228885013017, 89),
 (0.8564973941930835, 5774),
 (0.8564973941930835, 743),
 (0.8203828312573851, 8727)]

In [18]:
_arr = []
for x in arr[:5]:
    _arr.append(x[1])
_arr

[7033, 89, 5774, 743, 8727]

In [21]:
for idx in _arr:
    print(movies.iloc[idx])

movieId                             68848
title          Brothers Bloom, The (2008)
genres     Adventure|Comedy|Crime|Romance
Name: 7033, dtype: object
movieId                               101
title                Bottle Rocket (1996)
genres     Adventure|Comedy|Crime|Romance
Name: 89, dtype: object
movieId                                             31367
title                                   Chase, The (1994)
genres     Action|Adventure|Comedy|Crime|Romance|Thriller
Name: 5774, dtype: object
movieId                                     970
title                     Beat the Devil (1953)
genres     Adventure|Comedy|Crime|Drama|Romance
Name: 743, dtype: object
movieId                                              127052
title      Operation 'Y' & Other Shurik's Adventures (1965)
genres                                 Comedy|Crime|Romance
Name: 8727, dtype: object


In [17]:
print("Top 5 suggestions:")
for i in range(5):
    print('\t', movies.iloc[arr[i][1]].title)

Top 5 suggestions:
	 Brothers Bloom, The (2008)
	 Bottle Rocket (1996)
	 Chase, The (1994)
	 Beat the Devil (1953)
	 Operation 'Y' & Other Shurik's Adventures (1965)
